In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## MLP モデルのKerasによる実装
基礎編で使った2次元データを基に、MLPモデルをTensorflow/Kerasで書いてみます。

In [ ]:

# 二次元ガウス分布と一様分布
def get_dataset_2():
    from numpy.random import default_rng
    rng = default_rng(seed=0)  # 今回はデータセットの乱数を固定させます。

    num_signal = 100  # 生成するシグナルイベントの数
    num_background = 1000  # 生成するバックグラウンドイベントの数

    # データ点の生成
    ## 平均(x1,x2) = (1.0, 0.0)、分散=1の２次元ガウス分布
    x_sig = rng.multivariate_normal(mean=[1.0, 0],
                                    cov=[[1, 0], [0, 1]],
                                    size=num_signal)
    t_sig = np.ones((num_signal, 1))  # Signalは1にラベリング

    ## (-5, +5)の一様分布
    x_bg = rng.uniform(low=-5, high=5, size=(num_background, 2))
    t_bg = np.zeros((num_background, 1))  # Backgroundは0にラベリング

    # 2つのラベルを持つ学習データを1つにまとめる
    x = np.concatenate([x_sig, x_bg])
    t = np.concatenate([t_sig, t_bg])

    # データをランダムに並び替える
    p = rng.permutation(len(x))
    x, t = x[p], t[p]

    return x, t


# ラベル t={0,1}を持つデータ点のプロット
def plot_datapoint(x, t):
    # シグナル/バックグラウンドの抽出
    xS = x[t[:, 0] == 1]  # シグナルのラベルだけを抽出
    xB = x[t[:, 0] == 0]  # バックグラウンドのラベルだけを抽出

    # プロット
    plt.scatter(xS[:, 0], xS[:, 1], label='Signal', c='red', s=10)  # シグナルをプロット
    plt.scatter(xB[:, 0], xB[:, 1], label='Background', c='blue', s=10)  # バックグラウンドをプロット
    plt.xlabel('x1')  # x軸ラベルの設定
    plt.ylabel('x2')  # y軸ラベルの設定
    plt.legend()  # legendの表示
    plt.show()


# prediction関数 の等高線プロット (fill)
def plot_prediction(prediction, *args):
    # 等高線を描くためのメッシュの生成
    x1, x2 = np.mgrid[-5:5:100j, -5:5:100j]  # x1 = (-5, 5), x2 = (-5, 5) の範囲で100点x100点のメッシュを作成
    x1 = x1.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape(10000, ))
    x2 = x2.flatten()  # 二次元配列を一次元配列に変換 ( shape=(100, 100) => shape(10000, ))
    x = np.array([x1, x2]).T

    #  関数predictionを使って入力xから出力yを計算し、等高線プロットを作成
    y = prediction(x, *args)
    cs = plt.tricontourf(x[:, 0], x[:, 1], y.flatten(), levels=10)
    plt.colorbar(cs)


中間層が2層、それぞれの層のノード数がそれぞれ3つ、1つのMLPを構成します。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD

# データ点の取得
x, t = get_dataset_2()

# モデルの定義
model = Sequential([
    Input(shape=(2,)),
    Dense(units=3, activation='sigmoid'),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=1.0))

#  トレーニング
model.fit(
    x=x,
    y=t,
    batch_size=len(x),  # バッチサイズ。一回のステップで全てのデータを使うようにする。
    epochs=3000,  # 学習のステップ数
    verbose=0,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)


`Dense` は1層の隠れ層を作成する関数です。
`Dense`の詳細は[公式のドキュメント](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)を参照することでわかります。
ドキュメントを見ると、
```python
tf.keras.layers.Dense(
    units, activation=None, use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros', kernel_regularizer=None,
    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
    bias_constraint=None, **kwargs
)
```
のような引数を持つことがわかります。また、各引数の意味は、
* `units`:	Positive integer, dimensionality of the output space.
* `activation`:	Activation function to use. If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).
* `use_bias`:	Boolean, whether the layer uses a bias vector.
* `kernel_initializer`:	Initializer for the kernel weights matrix.
* `bias_initializer`:	Initializer for the bias vector.
* `kernel_regularizer`:	Regularizer function applied to the kernel weights matrix.
* `bias_regularizer`:	Regularizer function applied to the bias vector.
* `activity_regularizer`:	Regularizer function applied to the output of the layer (its "activation").
* `kernel_constraint`:	Constraint function applied to the kernel weights matrix.
* `bias_constraint`:	Constraint function applied to the bias vector.

のようになっています。隠れ層のノード数、重みの初期化方法、正規化方法、制約方法などを指定できることがわかります。
知らない関数を使うときは、必ずドキュメントを読んで、関数の入出力、引数、デフォルトの値などを確認するようにしましょう。
例えばこのDense関数は
```python
Dense(units=10)
```
のように、`units`(ノード数)だけを指定すれば動作しますが、その場合、暗に活性化関数は適用されず、重みの初期化は`glorot_uniform`で行われます。

`input_dim`は最初の層だけに対して必要となります。


Keras Model (上の例では`model`)は`summary`関数を使用することで、その構成が確認できます。

In [ ]:
model.summary()

このモデルは、１層目の隠れ層の出力が3, 学習可能なパラメータ数が9, 2層目の隠れ層の出力が1, 学習可能なパラメータ数が4 であることがわかります。"Output Shape"の"None"はサイズが未確定であることを表しています。ここでは、バッチサイズ用の次元になります。

モデルの構成図を作ってくれる便利なAPIも存在します。

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

層の数を増やしてみましょう。新たな層を重ねることで層の数を増やすことができます。
```python
model = Sequential([
    Dense(units=3, activation='sigmoid', input_dim=2),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=3, activation='sigmoid')  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=3, activation='sigmoid')  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])
```

In [ ]:
model = Sequential([
    Input(shape=(2,)),
    Dense(units=3, activation='sigmoid'),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=3, activation='sigmoid'),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=3, activation='sigmoid'),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=3, activation='sigmoid'),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=3, activation='sigmoid'),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

model.summary()

モデルのパラメータの数が増えていることがわかります。

次に、ノードの数を増やしてみましょう。

In [ ]:
model = Sequential([
    Input(shape=(2,)),
    Dense(units=128, activation='sigmoid'),  # ノード数が3の層を追加。活性化関数はシグモイド関数。
    Dense(units=128, activation='sigmoid'),  # ノード数が128の層を追加。活性化関数はシグモイド関数。
    Dense(units=128, activation='sigmoid'),  # ノード数が128の層を追加。活性化関数はシグモイド関数。
    Dense(units=128, activation='sigmoid'),  # ノード数が128の層を追加。活性化関数はシグモイド関数。
    Dense(units=128, activation='sigmoid'),  # ノード数が128の層を追加。活性化関数はシグモイド関数。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

model.summary()

パラメータの数が大きく増えたことがわかります。
MLPにおいては、パラメータの数は、ノード数の2乗で増加します。

このモデルを使って学習させてみましょう。

In [ ]:
#  誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01))

#  トレーニング
model.fit(
    x=x,
    y=t,
    batch_size=len(x),  # バッチサイズ。一回のステップで全てのデータを使うようにする。
    epochs=3000,  # 学習のステップ数
    verbose=0,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)

これまでは活性化関数としてシグモイド関数(`sigmoid`)を使っていました。昔はsigmoid関数やtanh関数がよく使われていましたが、最近はReLU関数がよく使われます。
$$
  ReLU = \begin{cases}
    x & (x \geq 0) \\
    0 & (x < 0)
  \end{cases}
$$

ReLUが好まれる理由については、別の資料を参照してください。

ReLUを使って学習がどのようになるか確認してみましょう。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# データ点の取得
x, t = get_dataset_2()

# モデルの定義
model = Sequential([
    Input(shape=(2,)),
    Dense(units=128, activation='relu'),  # ノード数が3の層を追加。活性化関数はReLU。
    Dense(units=128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU。
    Dense(units=128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU。
    Dense(units=128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU。
    Dense(units=128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01))

#  トレーニング
model.fit(
    x=x,
    y=t,
    batch_size=len(x),  # バッチサイズ。一回のステップで全てのデータを使うようにする。
    epochs=3000,  # 学習のステップ数
    verbose=0,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)

深層学習をトレーニングするにあたって、最適化関数(optimizer)も非常に重要な要素です。
確率的勾配降下法(SGD)の他によく使われるアルゴリズムとして adam があります。
adamを使ってみると、どのようになるでしょうか。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# データ点の取得
x, t = get_dataset_2()

# モデルの定義
model = Sequential([
    Input(shape=(2,)),
    Dense(units=128, activation='relu'),  # ノード数が3の層を追加。活性化関数はReLU。
    Dense(units=128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU。
    Dense(units=128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU。
    Dense(units=128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU。
    Dense(units=128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
model.compile(loss='binary_crossentropy', optimizer='adam')

#  トレーニング
model.fit(
    x=x,
    y=t,
    batch_size=len(x),  # バッチサイズ。一回のステップで全てのデータを使うようにする。
    epochs=3000,  # 学習のステップ数
    verbose=0,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# プロット
## パーセプトロンの出力を等高線プロット
plot_prediction(model.predict)

## データ点をプロット
plot_datapoint(x, t)

## Keras モデルの定義方法
Kerasモデルを定義する方法はいくつかあります。
最も簡単なのが`Sequential`を使った方法で、これまでの例では全てこの方法でモデルを定義してきました。
一方で、少し複雑なモデルを考えると、`Sequential`モデルで対応できなくなってきます。
一例としてResidual Network(ResNet)で使われるskip connectionを考えてみます。
skip connectionは
$$
y = f_2(f_1(x) + x)
$$
のように、入力を２つの経路に分け、片方はMLP、もう片方はそのまま後ろのレイヤーに接続するつなげ方です。
このようなモデルは、途中入出力の分岐があるため、`Sequential`モデルでは実装できません。
かわりに`Function API`を使うとこれを実装することができます。

`Functional API`では以下のようにしてモデルを定義します。

In [ ]:
from tensorflow.keras import Input, Model
input = Input(shape=(2,))
x = Dense(units=128, activation='relu')(input)
x = Dense(units=128, activation='relu')(x)
x = Dense(units=128, activation='relu')(x)
x = Dense(units=128, activation='relu')(x)
x = Dense(units=128, activation='relu')(x)
output = Dense(units=1, activation='sigmoid')(x)
model = Model(input, output)

入力(`Input`)をモジュールに順々に適用していき、
```python
x = Dense()(x)
```
最終的な出力(`output`)とはじめの入力を使って`Model`クラスを定義する、という流れになっています。

`Functional API`でskip connectionを実装すると、以下のようになります。

In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Add
input = Input(shape=(2,))
x = Dense(units=128, activation='relu')(input)
z = Dense(units=128, activation='relu')(x)
x = Add()([x, z])
z = Dense(units=128, activation='relu')(x)
x = Add()([x, z])
output = Dense(units=1, activation='sigmoid')(x)
model = Model(input, output)

from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

Kerasモデルを定義する方法として、`Model`クラスのサブクラスを作る方法もあります。
`Model`クラスをカスタマイズすることができるので、特殊な学習をさせたいときなど、高度な深層学習モデルを扱うときに使われることもあります。

In [ ]:
# Modelクラスを継承して新しいクラスを作成します
from tensorflow.keras import Model
class myModel(Model):
    def __init__(self):
        super().__init__()
        self.dense_1 = Dense(units=128, activation='relu')
        self.dense_2 = Dense(units=128, activation='relu')
        self.dense_3 = Dense(units=128, activation='relu')
        self.dense_4 = Dense(units=128, activation='relu')
        self.dense_5 = Dense(units=128, activation='relu')
        self.dense_6 = Dense(units=1, activation='sigmoid')

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        x = self.dense_4(x)
        x = self.dense_5(x)
        x = self.dense_6(x)
        return x

model = myModel()
